# Calendar generator

This script generates a calendar to be imported primarily to Google Calendar. Main features:

- List the matches of the selected competition and team
- Generates an output
- played matches has a score. It is also displayed.

In [1]:
#import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
import arrow
from ics import Calendar, Event

# Setting up parameters

In [2]:
#match_id = '76648'
#match_id = '75806'
#match_url = 'http://www.waterpolo.hu/meccs/' + match_id

competition_id = '680' # Fiú országos bajnokság
#competition_id = '704' # Gyermek I. Budapest bajnokság
competition_url = 'https://waterpolo.hu/bajnoksagok/?szures[bajnoksag_id]=' + competition_id

selected_team = "KSI"

# Parsing competition using BeutifulSoup

In [3]:
page = requests.get(competition_url, timeout=10)
#soup = BeautifulSoup(page.content, 'html-parser')
soup = BeautifulSoup(page.content, 'html5lib')
matches = soup.find_all("tr", class_='gamerow')


In [4]:
competition = list()

for match in matches:
    mtch = {'teams': []}
    competition.append(mtch)
    mtch['rawdata'] = mtch
    location = match.td
    for i, row in enumerate(match.find_all('td')):
        if row.text.startswith("202"):
            mtch['date_str'] = row.text.strip()
        elif i == 6 and row.text.strip() != '':
            mtch['location'] = row.text.strip()
    for link in match.find_all("a"):
        href = link['href']
        if '/csapat/'  in href:
            mtch['teams'].append(link.text)
        elif '/MatchReturnData' in href:
            mtch['report_url'] = href
        elif '/meccs/' in href:
            mtch['match_url'] = href
            mtch['results'] = link.text
        elif 'www.google.com/calendar/event' in href:
            mtch['google_calendar'] = href
        else:
            print(' nem feldolgozott sor: ', link)
random.sample(competition, 5)

[{'teams': [], 'rawdata': {...}},
 {'teams': ['Oázis SC', 'Aquaworm'],
  'rawdata': {...},
  'date_str': '2022. szept. 25. 10:00',
  'location': 'Szőnyi út 50-es sátor',
  'google_calendar': 'http://www.google.com/calendar/event?action=TEMPLATE&text=Fi%C3%BA+Gyermek+Orsz%C3%A1gos+Bajnoks%C3%A1g%3A+O%C3%A1zis+SC+-+Aquaworm&dates=20220925T100000/20220925T113000&sprop=Weboldal:http://waterpolo.hu&details=Fi%C3%BA+Gyermek+Orsz%C3%A1gos+V%C3%ADzilabda+Bajnoks%C3%A1g%0AAlapszakasz+-+A2+-+Keleti+csoport++-+1.+fordul%C3%B3++-+2022-09-25+10%3A00%0AO%C3%A1zis+SC+-+Aquaworm+%28Sz%C5%91nyi+%C3%BAt+50-es+s%C3%A1tor%29%0AJ%C3%A1t%C3%A9kvezet%C5%91k%3A+Csizmadia+Zsolt%2C+N%C3%A9meth+Anna&location=Sz%C5%91nyi+%C3%BAt+50-es+s%C3%A1tor&trp=true',
  'match_url': '/meccs/76239',
  'results': '28-8',
  'report_url': '/store_pdf/MatchReturnData_680_76239.pdf'},
 {'teams': ['KESI', 'Blamax Kaposvári VK'],
  'rawdata': {...},
  'date_str': '2022. okt. 09. 12:15',
  'location': 'Kecskemét',
  'google_calendar'

## Cleaning data

In [5]:
for match in competition:
    if len(match) < 5:
        del match

def filter_competition_by_team(competition: list, team: str) -> list:
    filtered = list()
    for match in competition:
        if len(match) > 3:
            for t in match['teams']:
                if team in t:
                    filtered.append(match)
                    break
    return filtered

competition_ksi = filter_competition_by_team(competition, "KSI")

for i, match in enumerate(competition_ksi):
    print(i, match['date_str'], match['teams'], match['results'])

0 2022. szept. 25. 11:15 ['KSI SE', 'KESI'] 18-6
1 2022. okt. 02. 12:15 ['Blamax Kaposvári VK', 'KSI SE'] 1-26
2 2022. okt. 09. 15:15 ['KSI SE', 'Legrand Szentes'] 21-4
3 2022. okt. 12. 17:00 ['Honvéd', 'KSI SE'] 2-23
4 2022. okt. 16. 11:30 ['AVUS', 'KSI SE'] 1-26
5 2022. okt. 30. 12:15 ['KSI SE', 'CPVSE'] 23-1
6 2022. nov. 06. 15:15 ['KSI SE', 'Szolnoki Dózsa'] 7-6
7 2022. nov. 13. 10:00 ['Genesys OSC Újbuda', 'KSI SE'] 7-19
8 2022. nov. 20. 12:15 ['KSI SE', 'Pécsi Sportiskola'] 9-8
9 2022. nov. 27. 12:15 ['Tigra-Zf-Eger', 'KSI SE'] 0-0
10 2022. dec. 04. 11:15 ['KSI SE', 'Vasas SC'] 20-7
11 2022. dec. 11. 12:30 ['FTC-Telekom', 'KSI SE'] 7-11
12 2022. dec. 18. 11:15 ['KSI SE', 'UVSE I.'] 7-13
13 2023. febr. 05. 10:00 ['SZVPS', 'KSI SE'] 0-0
14 2023. febr. 12. 11:15 ['KSI SE', 'BVSC-Zugló'] 0-0


## Generating calendar

In [6]:
calendar = Calendar()

for match in competition_ksi:
    event = Event()
    event.name = f'{match["teams"][0]} - {match["teams"][1]}: {match["results"]}'
    
    event.begin = arrow.get(match['date_str'], "YYYY. MMM. D. H:mm", locale= "hu")
    event.duration = {'hours':1}

    event.location = match['location']
    calendar.events.add(event)

print(calendar.serialize())

BEGIN:VCALENDAR
VERSION:2.0
PRODID:ics.py - http://git.io/lLljaA
BEGIN:VEVENT
DURATION:PT1H
LOCATION:Nyéki I. - 33m-es
DTSTART:20221113T100000Z
SUMMARY:Genesys OSC Újbuda - KSI SE: 7-19
UID:66e8e782-a895-47a7-8fad-b609a6959594@66e8.org
END:VEVENT
BEGIN:VEVENT
DURATION:PT1H
LOCATION:Komjádi
DTSTART:20221009T151500Z
SUMMARY:KSI SE - Legrand Szentes: 21-4
UID:8a0e17f2-83ab-406a-be20-45cf5c2fa8ef@8a0e.org
END:VEVENT
BEGIN:VEVENT
DURATION:PT1H
LOCATION:Kőér u.
DTSTART:20221012T170000Z
SUMMARY:Honvéd - KSI SE: 2-23
UID:f0a5f457-d3bc-414a-aac8-dd586a8b874c@f0a5.org
END:VEVENT
BEGIN:VEVENT
DURATION:PT1H
LOCATION:Széchy T. u. Margitsz.
DTSTART:20221204T111500Z
SUMMARY:KSI SE - Vasas SC: 20-7
UID:7a631d7c-9d8d-4e48-99c5-f67b36076ae4@7a63.org
END:VEVENT
BEGIN:VEVENT
DURATION:PT1H
LOCATION:Széchy T. u. Margitsz.
DTSTART:20230212T111500Z
SUMMARY:KSI SE - BVSC-Zugló: 0-0
UID:65b66462-81c4-474d-8ff2-1fc00893eb9c@65b6.org
END:VEVENT
BEGIN:VEVENT
DURATION:PT1H
LOCATION:Széchy T. u. Margitsz.
DTSTART:20